# 2024/08/26 PRISM ADRIO Demo
_Author: Meaghan Freund_

### **Background**

PRISM (Parameter-elevation Regressions on Independent Slopes Model) is an Oregon state website that provides users with datasets covering observed climate variables across the 48 conterminous United States. The recorded values are displayed as a 4km resolution grid across the previously mentioned map, with a value of the climate attribute for each 4km square. For the climate variables, PRISM produces data on: 

- Precipitation (mm)
- Minimum Temperature (°C)
- Mean Temperature (°C)
- Maximum Temperature (°C)
- Minimum Vapor Pressure Deficit (hPa)
- Maximum Vapor Pressure Deficit (hPa)
- Mean Dew Point Temperature (°C)

On the PRISM website, users are able to generate raster, or pixelated, graphics of recorded observations on a daily, monthly, or annual basis for any of these climate attributes. PRISM provides readable datasets as zip files for the monthly and daily data, being one zip file per variable and for each whole month or day. 

For more information, visit the PRISM website homepage: _https://prism.oregonstate.edu/_

### **Simple Query**

For a finer look at the raster data PRISM provides, the PRISM ADRIO template provides data within the daily granularity. For a basic example, a user may look at a single day with one climate variable. The "Precipitation" and the "Mean Dew Point Temperature" calls are quite simple as they exclude having a maximum or minimum, unlike the other calls. The below call uses the four provided counties and queries for June 1st, 2024.

#### Precipitation

In [1]:
from epymorph.geography.us_census import CountyScope
from epymorph import *
from epymorph.adrio import acs5, us_tiger
from epymorph.simulator.data import evaluate_param
from datetime import date
from epymorph.adrio import prism


county_scope = CountyScope.in_counties(["04013", "08041", "32003", "35001"])
time_period = TimeFrame.range(date(2024, 6, 1), date(2024, 6, 1))
precipitation = prism.Precipitation(time_period)
geoids = county_scope.get_node_ids()

rume = SingleStrataRume.build(
    ipm_library["no"](),
    mm_library["no"](),
    init.NoInfection(),
    scope=county_scope,
    time_frame=time_period,
    params={
        "population": acs5.Population(),
        "centroid": us_tiger.GeometricCentroid(),
        "precipitation": prism.Precipitation(time_period),
    },
)
with sim_messaging():
    precipitation = evaluate_param(rume, "precipitation")


Loading epymorph.adrio.us_tiger.GeometricCentroid:
  |####################| 100%  (11.971s)
Loading epymorph.adrio.prism.Precipitation:
  |####################| 100%  (0.227s)


In [3]:
print(f"Counties: {geoids}\n")

print(f"Precipitation in mm:\n {precipitation}\n")

Counties: ['04013' '08041' '32003' '35001']

Precipitation in mm:
 [[0.         3.52799988 0.         0.        ]]



### **Date Range**

The above query calls for a "date range" of June 1st, 2024 to that same day, simply calling for that single day. The ADRIO template here can call from the beginning of the month to the end of that same month (may test to see if it can cross months). Changing the second date called will grab the climate data in between each given date inclusively.

### **Minimum, Maximum, and Mean**

All of the observed climate variables are recorded differently, with some attributes providing a mean, minimum, or maximum. Precipitation is the only variable that does not record any of these, with the measure only being provided as a total accumulation. Below is the list of calls that are available to a user, with "Precipitation" and "Dew Point Temperature" having a single call type, as there are no other options. 

- Precipitation
    - total accumulated

- Dew Point Temperature
    - mean

- Temperature
    - mean
    - minimum
    - maximum

- Vapor Pressure Deficit
    - minimum
    - maximum

When calling "Temperature" or "Vapor Pressure Deficit", specifying "Mean", "Minimum", or "Maximum" within the PRISM call will provide the desired information.

#### **Temperature for 30 Days**

In the below instance, it is specified to get the mean temperature for the whole month of April in 2021.

In [2]:
from epymorph.geography.us_census import StateScope
from epymorph import *
from epymorph.adrio import acs5, us_tiger
from epymorph.simulator.data import evaluate_param
from datetime import date
from epymorph.adrio import prism

state_scope = StateScope.in_states_by_code(["AZ"])
time_period = TimeFrame.range(date(2021, 4, 1), date(2021, 4, 30))
geoids = state_scope.get_node_ids()

rume = SingleStrataRume.build(
    ipm_library["no"](),
    mm_library["no"](),
    init.NoInfection(),
    scope=state_scope,
    time_frame=time_period,
    params={
        "temperature": prism.Temperature(time_period, "Mean"),
        "population": acs5.Population(),
        "centroid": us_tiger.GeometricCentroid(),
    },
)
with sim_messaging():
    temperature = evaluate_param(rume, "temperature")

Loading epymorph.adrio.us_tiger.GeometricCentroid:
  |####################| 100%  (3.046s)
Loading epymorph.adrio.prism.Temperature:
  |####################| 100%  (22.394s)


In [3]:
print(f"Counties: {geoids}\n")

print(f"Temperature in Celsius:\n {temperature}\n")

Counties: ['04']

Temperature in Celsius:
 [[18.10199928]
 [20.5033989 ]
 [21.53979874]
 [23.15460014]
 [23.13929939]
 [22.01089859]
 [19.28979874]
 [19.92639923]
 [20.50819969]
 [20.42409897]
 [19.83799934]
 [19.71660042]
 [20.21759987]
 [18.79469872]
 [18.23429871]
 [16.08849907]
 [16.52569962]
 [17.06690025]
 [17.06819916]
 [17.9034996 ]
 [18.67819977]
 [18.19149971]
 [16.32390022]
 [17.16259956]
 [18.77140045]
 [19.666399  ]
 [15.94489956]
 [13.77489948]
 [15.1602993 ]
 [20.76469994]]

